# Операторы градиента и детекция границ

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import time
from scipy import ndimage
import os

In [ ]:
# Загрузка тестового изображения
img_path = 'sample.jpg'

if os.path.exists(img_path):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    print(f"✅ Изображение загружено: {img_path}")
else:
    print("📝 Создаём тестовое изображение для демонстрации детекции границ...")

    # Создаём изображение с чёткими границами для демонстрации
    img = np.zeros((400, 500, 3), dtype=np.uint8)

    # Геометрические фигуры с разными типами границ
    # Прямоугольники
    img[50:150, 50:200] = [200, 200, 200]   # Светло-серый
    img[200:300, 50:200] = [100, 100, 100]  # Тёмно-серый

    # Круги
    cv2.circle(img, (350, 100), 50, (150, 150, 150), -1)  # Заполненный круг
    cv2.circle(img, (350, 250), 40, (50, 50, 50), 3)      # Контур круга

    # Линии разной толщины и ориентации
    cv2.line(img, (100, 320), (400, 350), (180, 180, 180), 2)  # Диагональ
    cv2.line(img, (250, 50), (250, 350), (120, 120, 120), 4)   # Вертикаль

    # Добавляем немного шума для реалистичности
    noise = np.random.randint(-20, 20, img.shape, dtype=np.int16)
    img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    print("✅ Синтетическое изображение создано")

# Преобразуем в grayscale
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Отображаем исходное изображение
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.title('Исходное изображение (RGB)')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img_gray, cmap='gray')
plt.title('Grayscale версия')
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 1: Реализация операторов градиента "с нуля"
# ===============================

def apply_gradient_operator(image, kernel_x, kernel_y, name="Custom"):
    """Применяет оператор градиента с заданными ядрами"""

    # Применяем свёртку для X и Y направлений
    grad_x = cv2.filter2D(image, cv2.CV_32F, kernel_x)
    grad_y = cv2.filter2D(image, cv2.CV_32F, kernel_y)

    # Вычисляем магнитуду и направление
    magnitude = np.sqrt(grad_x**2 + grad_y**2)
    direction = np.arctan2(grad_y, grad_x)

    # Конвертируем в uint8 для отображения
    magnitude_uint8 = cv2.convertScaleAbs(magnitude)

    return grad_x, grad_y, magnitude, direction, magnitude_uint8

# Определяем ядра различных операторов
operators = {
    'Sobel': {
        'x': np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32),
        'y': np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=np.float32)
    },
    'Prewitt': {
        'x': np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]], dtype=np.float32),
        'y': np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]], dtype=np.float32)
    },
    'Roberts': {
        'x': np.array([[1, 0], [0, -1]], dtype=np.float32),
        'y': np.array([[0, 1], [-1, 0]], dtype=np.float32)
    }
}

# Лапласиан (изотропный оператор второй производной)
laplacian_kernels = {
    'Laplacian 4-connected': np.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]], dtype=np.float32),
    'Laplacian 8-connected': np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]], dtype=np.float32)
}

# Применяем все операторы градиента
gradient_results = {}

for name, kernels in operators.items():
    print(f"🔄 Применяем оператор {name}...")
    grad_x, grad_y, magnitude, direction, mag_uint8 = apply_gradient_operator(
        img_gray, kernels['x'], kernels['y'], name
    )
    gradient_results[name] = {
        'grad_x': grad_x,
        'grad_y': grad_y,
        'magnitude': magnitude,
        'magnitude_uint8': mag_uint8,
        'direction': direction
    }

# Применяем операторы Лапласиана
laplacian_results = {}
for name, kernel in laplacian_kernels.items():
    print(f"🔄 Применяем {name}...")
    laplacian = cv2.filter2D(img_gray, cv2.CV_32F, kernel)
    laplacian_uint8 = cv2.convertScaleAbs(laplacian)
    laplacian_results[name] = {
        'raw': laplacian,
        'uint8': laplacian_uint8
    }

# Визуализируем результаты операторов градиента
fig, axes = plt.subplots(3, 4, figsize=(16, 12))

# Исходное изображение
axes[0, 0].imshow(img_gray, cmap='gray')
axes[0, 0].set_title('Исходное изображение')
axes[0, 0].axis('off')

# Результаты операторов градиента
for i, (name, result) in enumerate(gradient_results.items(), 1):
    if i < 4:
        axes[0, i].imshow(result['magnitude_uint8'], cmap='gray')
        axes[0, i].set_title(f'{name} - Магнитуда')
        axes[0, i].axis('off')

# Компоненты X для операторов
for i, (name, result) in enumerate(gradient_results.items()):
    if i < 3:
        axes[1, i].imshow(result['grad_x'], cmap='RdBu')
        axes[1, i].set_title(f'{name} - Градиент X')
        axes[1, i].axis('off')

# Лапласианы
for i, (name, result) in enumerate(laplacian_results.items()):
    axes[1, 3].imshow(result['uint8'], cmap='gray') if i == 0 else axes[2, 0].imshow(result['uint8'], cmap='gray')
    (axes[1, 3] if i == 0 else axes[2, 0]).set_title(name)
    (axes[1, 3] if i == 0 else axes[2, 0]).axis('off')

# Компоненты Y для операторов
for i, (name, result) in enumerate(gradient_results.items()):
    if i < 3:
        axes[2, i+1].imshow(result['grad_y'], cmap='RdBu')
        axes[2, i+1].set_title(f'{name} - Градиент Y')
        axes[2, i+1].axis('off')

plt.tight_layout()
plt.show()

# Сравнение с встроенными функциями OpenCV
print("\n📊 Сравнение с встроенными функциями OpenCV:")

# Sobel OpenCV
sobel_x_cv = cv2.Sobel(img_gray, cv2.CV_32F, 1, 0, ksize=3)
sobel_y_cv = cv2.Sobel(img_gray, cv2.CV_32F, 0, 1, ksize=3)
sobel_magnitude_cv = np.sqrt(sobel_x_cv**2 + sobel_y_cv**2)

# Laplacian OpenCV
laplacian_cv = cv2.Laplacian(img_gray, cv2.CV_32F)

# Сравниваем точность
sobel_diff = np.abs(gradient_results['Sobel']['magnitude'] - sobel_magnitude_cv)
laplacian_diff = np.abs(laplacian_results['Laplacian 4-connected']['raw'] - laplacian_cv)

print(f"Разность Sobel (наш vs OpenCV): макс={sobel_diff.max():.2f}, средн={sobel_diff.mean():.2f}")
print(f"Разность Laplacian (наш vs OpenCV): макс={laplacian_diff.max():.2f}, средн={laplacian_diff.mean():.2f}")

In [ ]:
# ===============================
# ЗАДАНИЕ 2: Направление градиента и его визуализация
# ===============================

def visualize_gradient_direction(grad_x, grad_y, magnitude, name="", magnitude_threshold=50):
    """Визуализирует направление градиента с помощью цветового кодирования"""

    # Вычисляем направление в градусах
    direction = np.arctan2(grad_y, grad_x) * 180 / np.pi
    direction[direction < 0] += 360  # Приводим к диапазону [0, 360]

    # Создаём цветовую карту направлений
    # Используем HSV: Hue = направление, Saturation = const, Value = magnitude
    hsv = np.zeros((grad_x.shape[0], grad_x.shape[1], 3), dtype=np.uint8)

    # Нормализуем направление для Hue канала (0-179 в OpenCV)
    hsv[:, :, 0] = (direction / 2).astype(np.uint8)
    hsv[:, :, 1] = 255  # Максимальная насыщенность

    # Value пропорциональна магнитуде
    magnitude_normalized = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    hsv[:, :, 2] = magnitude_normalized.astype(np.uint8)

    # Конвертируем HSV в RGB
    direction_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

    # Применяем порог магнитуды (показываем направления только для сильных градиентов)
    mask = magnitude > magnitude_threshold
    direction_rgb_masked = direction_rgb.copy()
    direction_rgb_masked[~mask] = [0, 0, 0]  # Чёрный для слабых градиентов

    return direction, direction_rgb, direction_rgb_masked

# Анализируем направления для оператора Собеля
sobel_result = gradient_results['Sobel']
direction_deg, direction_rgb, direction_masked = visualize_gradient_direction(
    sobel_result['grad_x'],
    sobel_result['grad_y'],
    sobel_result['magnitude'],
    "Sobel"
)

# Создаём круговую диаграмму направлений
def plot_direction_histogram(direction, magnitude, threshold=50):
    """Строит гистограмму направлений градиента"""

    # Фильтруем по порогу магнитуды
    strong_gradients = magnitude > threshold
    directions_filtered = direction[strong_gradients]

    # Создаём гистограмму направлений (в радианах для полярных координат)
    directions_rad = directions_filtered * np.pi / 180

    # Разбиваем на 36 бинов (по 10 градусов)
    bins = np.linspace(0, 2*np.pi, 37)
    hist, _ = np.histogram(directions_rad, bins)

    # Строим полярную гистограмму
    angles = bins[:-1]
    width = 2 * np.pi / len(hist)

    ax = plt.subplot(111, projection='polar')
    bars = ax.bar(angles, hist, width=width, alpha=0.7)

    # Цветовое кодирование
    colors = plt.cm.hsv(angles / (2 * np.pi))
    for bar, color in zip(bars, colors):
        bar.set_facecolor(color)

    ax.set_title('Распределение направлений градиента', pad=20)
    plt.show()

# Визуализируем результаты анализа направлений
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Исходное изображение
axes[0, 0].imshow(img_gray, cmap='gray')
axes[0, 0].set_title('Исходное изображение')
axes[0, 0].axis('off')

# Магнитуда градиента
axes[0, 1].imshow(sobel_result['magnitude'], cmap='gray')
axes[0, 1].set_title('Магнитуда градиента (Sobel)')
axes[0, 1].axis('off')

# Направление градиента (полная карта)
axes[0, 2].imshow(direction_rgb)
axes[0, 2].set_title('Направление градиента (HSV кодирование)')
axes[0, 2].axis('off')

# Направление градиента (с порогом)
axes[1, 0].imshow(direction_masked)
axes[1, 0].set_title('Направление (магнитуда > 50)')
axes[1, 0].axis('off')

# Градиент X
axes[1, 1].imshow(sobel_result['grad_x'], cmap='RdBu')
axes[1, 1].set_title('Градиент X')
axes[1, 1].axis('off')

# Градиент Y
axes[1, 2].imshow(sobel_result['grad_y'], cmap='RdBu')
axes[1, 2].set_title('Градиент Y')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

# Строим гистограмму направлений
print("📊 Гистограмма направлений градиента:")
plot_direction_histogram(direction_deg, sobel_result['magnitude'])

In [ ]:
# ===============================
# ЗАДАНИЕ 3: Алгоритм Canny - поэтапная реализация
# ===============================

def canny_step_by_step(image, low_threshold=50, high_threshold=150, kernel_size=5, sigma=1.4):
    """
    Поэтапная реализация алгоритма Canny
    """
    print("🔄 Выполняем алгоритм Canny поэтапно...")

    # Шаг 1: Подавление шума (Gaussian Blur)
    print("   Шаг 1: Подавление шума (Gaussian Blur)")
    blurred = cv2.GaussianBlur(image, (kernel_size, kernel_size), sigma)

    # Шаг 2: Вычисление градиента (Sobel)
    print("   Шаг 2: Вычисление градиента")
    grad_x = cv2.Sobel(blurred, cv2.CV_32F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(blurred, cv2.CV_32F, 0, 1, ksize=3)

    magnitude = np.sqrt(grad_x**2 + grad_y**2)
    direction = np.arctan2(grad_y, grad_x)

    # Шаг 3: Подавление не-максимумов (Non-maximum suppression)
    print("   Шаг 3: Подавление не-максимумов")
    suppressed = non_maximum_suppression(magnitude, direction)

    # Шаг 4: Двойная пороговая фильтрация и связывание границ
    print("   Шаг 4: Двойная пороговая фильтрация")
    edges = double_threshold_and_linking(suppressed, low_threshold, high_threshold)

    return {
        'original': image,
        'blurred': blurred,
        'grad_x': grad_x,
        'grad_y': grad_y,
        'magnitude': magnitude,
        'direction': direction,
        'suppressed': suppressed,
        'edges': edges
    }

def non_maximum_suppression(magnitude, direction):
    """Подавление не-максимумов"""

    rows, cols = magnitude.shape
    suppressed = np.zeros_like(magnitude)

    # Преобразуем направления в градусы
    angle = direction * 180.0 / np.pi
    angle[angle < 0] += 180

    for i in range(1, rows-1):
        for j in range(1, cols-1):
            q = 255
            r = 255

            # Определяем направление градиента
            if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                # Горизонтальное направление
                q = magnitude[i, j+1]
                r = magnitude[i, j-1]
            elif (22.5 <= angle[i,j] < 67.5):
                # Диагональ /
                q = magnitude[i+1, j-1]
                r = magnitude[i-1, j+1]
            elif (67.5 <= angle[i,j] < 112.5):
                # Вертикальное направление
                q = magnitude[i+1, j]
                r = magnitude[i-1, j]
            elif (112.5 <= angle[i,j] < 157.5):
                # Диагональ \
                q = magnitude[i-1, j-1]
                r = magnitude[i+1, j+1]

            # Сохраняем пиксель только если он локальный максимум
            if magnitude[i,j] >= q and magnitude[i,j] >= r:
                suppressed[i,j] = magnitude[i,j]
            else:
                suppressed[i,j] = 0

    return suppressed

def double_threshold_and_linking(image, low_threshold, high_threshold):
    """Двойная пороговая фильтрация и связывание границ"""

    rows, cols = image.shape
    edges = np.zeros_like(image, dtype=np.uint8)

    # Классификация пикселей
    strong_edges = (image > high_threshold)
    weak_edges = ((image >= low_threshold) & (image <= high_threshold))

    # Сильные границы = белые пиксели
    edges[strong_edges] = 255

    # Связывание слабых границ с сильными
    for i in range(1, rows-1):
        for j in range(1, cols-1):
            if weak_edges[i, j]:
                # Проверяем 8-связность со сильными границами
                if np.any(edges[i-1:i+2, j-1:j+2] == 255):
                    edges[i, j] = 255
                else:
                    edges[i, j] = 0

    return edges

# Применяем поэтапный Canny
canny_steps = canny_step_by_step(img_gray, low_threshold=50, high_threshold=150)

# Сравниваем с встроенной функцией OpenCV
canny_opencv = cv2.Canny(img_gray, 50, 150)

# Визуализируем все этапы
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Верхняя строка - этапы обработки
axes[0, 0].imshow(canny_steps['original'], cmap='gray')
axes[0, 0].set_title('1. Исходное изображение')
axes[0, 0].axis('off')

axes[0, 1].imshow(canny_steps['blurred'], cmap='gray')
axes[0, 1].set_title('2. После Gaussian Blur')
axes[0, 1].axis('off')

axes[0, 2].imshow(canny_steps['magnitude'], cmap='gray')
axes[0, 2].set_title('3. Магнитуда градиента')
axes[0, 2].axis('off')

axes[0, 3].imshow(canny_steps['suppressed'], cmap='gray')
axes[0, 3].set_title('4. После подавления не-максимумов')
axes[0, 3].axis('off')

# Нижняя строка - финальные результаты и сравнения
axes[1, 0].imshow(canny_steps['edges'], cmap='gray')
axes[1, 0].set_title('5. Наша реализация Canny')
axes[1, 0].axis('off')

axes[1, 1].imshow(canny_opencv, cmap='gray')
axes[1, 1].set_title('6. OpenCV Canny')
axes[1, 1].axis('off')

# Разность между реализациями
difference = np.abs(canny_steps['edges'].astype(float) - canny_opencv.astype(float))
axes[1, 2].imshow(difference, cmap='hot')
axes[1, 2].set_title('7. Разность реализаций')
axes[1, 2].axis('off')

# Направление градиента
direction_vis = (canny_steps['direction'] * 180 / np.pi + 180) / 360 * 255
axes[1, 3].imshow(direction_vis.astype(np.uint8), cmap='hsv')
axes[1, 3].set_title('8. Направление градиента')
axes[1, 3].axis('off')

plt.tight_layout()
plt.show()

print(f"📊 Сравнение реализаций Canny:")
print(f"Максимальная разность: {difference.max():.1f}")
print(f"Средняя разность: {difference.mean():.2f}")
print(f"Процент совпадающих пикселей: {np.mean(difference == 0) * 100:.1f}%")

In [ ]:
# ===============================
# ЗАДАНИЕ 4: Интерактивное исследование параметров Canny
# ===============================

def canny_parameter_study(image):
    """Исследование влияния различных параметров на результат Canny"""

    # Различные комбинации параметров
    parameter_sets = [
        {'low': 50, 'high': 100, 'blur': 1, 'title': 'Низкие пороги + слабое размытие'},
        {'low': 50, 'high': 150, 'blur': 1, 'title': 'Средние пороги + слабое размытие'},
        {'low': 100, 'high': 200, 'blur': 1, 'title': 'Высокие пороги + слабое размытие'},
        {'low': 50, 'high': 150, 'blur': 3, 'title': 'Средние пороги + среднее размытие'},
        {'low': 50, 'high': 150, 'blur': 5, 'title': 'Средние пороги + сильное размытие'},
        {'low': 30, 'high': 80, 'blur': 2, 'title': 'Чувствительные настройки'}
    ]

    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()

    for i, params in enumerate(parameter_sets):
        # Применяем предварительное размытие
        if params['blur'] > 1:
            blurred = cv2.GaussianBlur(image, (params['blur'], params['blur']), 0)
        else:
            blurred = image.copy()

        # Применяем Canny
        edges = cv2.Canny(blurred, params['low'], params['high'])

        # Отображаем результат
        axes[i].imshow(edges, cmap='gray')
        axes[i].set_title(f"{params['title']}\nLow={params['low']}, High={params['high']}, Blur={params['blur']}")
        axes[i].axis('off')

        # Подсчитываем статистику
        edge_pixels = np.sum(edges > 0)
        total_pixels = edges.shape[0] * edges.shape[1]
        edge_percentage = edge_pixels / total_pixels * 100

        print(f"📊 {params['title']}: {edge_percentage:.2f}% пикселей определены как границы")

    plt.tight_layout()
    plt.show()

# Проводим исследование параметров
canny_parameter_study(img_gray)

In [ ]:
# ===============================
# ЗАДАНИЕ 5: Сравнительный анализ всех методов детекции границ
# ===============================

def comprehensive_edge_detection_comparison(image):
    """Сравнивает различные методы детекции границ"""

    methods = {}

    # 1. Операторы градиента
    sobel_x = cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize=3)
    methods['Sobel'] = cv2.convertScaleAbs(np.sqrt(sobel_x**2 + sobel_y**2))

    prewitt_x = cv2.filter2D(image, cv2.CV_32F, np.array([[-1,0,1],[-1,0,1],[-1,0,1]]))
    prewitt_y = cv2.filter2D(image, cv2.CV_32F, np.array([[-1,-1,-1],[0,0,0],[1,1,1]]))
    methods['Prewitt'] = cv2.convertScaleAbs(np.sqrt(prewitt_x**2 + prewitt_y**2))

    # 2. Лапласиан
    methods['Laplacian'] = cv2.convertScaleAbs(cv2.Laplacian(image, cv2.CV_32F))

    # 3. Лапласиан Гаусса (LoG)
    gaussian = cv2.GaussianBlur(image, (5, 5), 1.4)
    methods['LoG'] = cv2.convertScaleAbs(cv2.Laplacian(gaussian, cv2.CV_32F))

    # 4. Canny (различные настройки)
    methods['Canny (50,150)'] = cv2.Canny(image, 50, 150)
    methods['Canny (100,200)'] = cv2.Canny(image, 100, 200)

    return methods

# Сравниваем все методы
edge_methods = comprehensive_edge_detection_comparison(img_gray)

# Визуализируем результаты
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

# Исходное изображение
axes[0].imshow(img_gray, cmap='gray')
axes[0].set_title('Исходное изображение')
axes[0].axis('off')

# Результаты различных методов
for i, (name, result) in enumerate(edge_methods.items(), 1):
    if i < len(axes):
        axes[i].imshow(result, cmap='gray')
        axes[i].set_title(name)
        axes[i].axis('off')

        # Подсчитываем статистику
        if 'Canny' in name:
            edge_pixels = np.sum(result > 0)
        else:
            # Для градиентных методов применяем порог
            threshold = np.mean(result) + 2 * np.std(result)
            edge_pixels = np.sum(result > threshold)

        total_pixels = result.shape[0] * result.shape[1]
        edge_percentage = edge_pixels / total_pixels * 100
        print(f"📊 {name}: {edge_percentage:.2f}% пикселей определены как границы")

plt.tight_layout()
plt.show()

# ДОМАШНЕЕ ЗАДАНИЕ

1. Реализуйте адаптивный алгоритм Canny, который автоматически подбирает пороги на основе анализа гистограммы градиентов

2. Создайте функцию для детекции границ определённой ориентации (например, только вертикальных или горизонтальных линий)

3. Исследуйте влияние различных типов шума на качество детекции границ
